In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv("..\data\ElectricityMap_DE_2021.csv", sep=",")
df = df[["datetime","power_production_wind_avg"]]

# seperate datetime into hour of day, day of year, and month of year
df["datetime"] = pd.to_datetime(df["datetime"])
df["hour_of_day"] = df["datetime"].dt.hour
df["day_of_year"] = df["datetime"].dt.day_of_year
df["month_of_year"] = df["datetime"].dt.month

# rescale data (min-max normalization)
df_norm = df.drop("datetime", axis=1)
df_norm = (df_norm-df_norm.min())/(df_norm.max()-df_norm.min())

# add sin time columns (range: 0-1)
df_sin = df_norm.drop("power_production_wind_avg", axis=1)
df_sin = df_sin.apply(lambda x: np.sin(2*np.pi*x))
df_sin = df_sin.add_prefix("sin_")

# add cos time columns (range: 0-1)
df_cos = df_norm.drop("power_production_wind_avg", axis=1)
df_cos = df_cos.apply(lambda x: np.cos(2*np.pi*x))
df_cos = df_cos.add_prefix("cos_")

# merge columns
y = pd.concat([df_norm, df_sin,df_cos], axis=1)

y.tail()